###  Webdriver를 이용한 크롤링

<p align = "center">
  <img src = "https://user-images.githubusercontent.com/76269316/164836248-a14323f3-e78d-4b5c-afbc-f466864623de.png">
</p>
<p align = "center">
  네이버 뉴스 페이지
</p>


<p align = "center">
  <img src = "https://user-images.githubusercontent.com/76269316/164833886-5216bdec-6d3f-4690-8675-fca79a114966.png">
</p>
<p align = "center">
  삼성전자 검색 결과
</p>

네이버 뉴스 페이지에서 오른쪽 상단에 있는 검색창을 통해 관련 기사를 검색하면 아래 페이지로 이동한다. <br>

아래 페이지로 이동시 나오는 기사들은 해당 신문사의 페이지로 이동하기 때문에 기사 내용들이 모두 다른 포맷으로 작성돼 있다.

<br>

<img width="1624" alt="스크린샷 2022-04-23 오전 9 06 46" src="https://user-images.githubusercontent.com/76269316/164836426-7ac782e5-73b6-4bec-99af-2516c989a65f.png">

<br>

<img width="1624" alt="스크린샷 2022-04-23 오전 9 08 10" src="https://user-images.githubusercontent.com/76269316/164836482-02bf3bce-9d59-4ae3-b090-0a3a2e0d6b0c.png">

### Naver API를 이용한 크롤링

<p align = "center">
  <img src = "https://user-images.githubusercontent.com/76269316/164836860-acaf96f9-24d1-4293-b798-c49dfd16b45f.png">
</p>
<p align = "center">
  네이버 뉴스 API를 통해 크롤링한 뉴스 링크
</p>

<br>

<p align = "center">
  <img src = "https://user-images.githubusercontent.com/76269316/164836977-6d162096-a293-4053-96bc-23b63f7dac7b.png">
</p>
<p align = "center">
  크롤링한 뉴스 기사 (네이버)
</p>

<p align = "center">
  <img src = "https://user-images.githubusercontent.com/76269316/164978236-0f75dd28-c9d0-47ce-9212-7f572f1f1c15.png">
</p>
<p align = "center">
  기사 본문에 있는 네이버
</p>

<br>

API를 통해 검색된 기사들은 네이버에서 제공하는 포맷으로 보여지기 때문에 크롤링하기엔 적합하지만, 내용상 연관성이 떨어지는 기사들이 너무 많다.


In [1]:
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 브라우저를 띄우지 않음
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [2]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome('chromedriver', options=chrome_options)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
import re

def cleansing(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail주소제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]' # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.replace('\n',"")  # 개행 문자 제거
    text = text.replace('\t',"")
    return text    

In [4]:
# company_code(종목번호), maxpage(크롤링 할  페이지 수), title(제목이 연관된 기사 - True, 제목 + 내용이 연관된 기사 - False) 
def news_crawling(company_code, maxpage, title=True):
    page = 1

    title_list = []
    article_list = []
    link_list = []
    date_list = []
    newspaper_list = []

    while page <= int(maxpage):
        print('page: ', page)
        if title:  # 제목
            url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page)
        else:  # 제목 + 내용
            url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page) + '&sm=entity_id.basic'

        html = requests.get(url).text
        html = BeautifulSoup(html, 'lxml')

        # 뉴스 제목
        titles = html.select('.title')
        for title in titles:
            title_list.append(re.sub('\n', '',  title.get_text()))
            
        # 기사 내용
        links = html.select('.title')
        
        # 기사 링크 parsing
        url_list = []
        for link in links:
            url_list.append('https://finance.naver.com' + link.find('a')['href'])
            link_list.append('https://finance.naver.com' + link.find('a')['href'])

        for article_url in url_list:
            article_list.append(get_article(article_url))

        
        # 뉴스 날짜
        dates = html.select('.date')
        for date in dates:
            date_list.append(date.get_text())

        # 신문사
        newspapers = html.select('.info')
        for newspaper in newspapers:
            newspaper_list.append(newspaper.get_text())
            
        print(len(title_list),  len(link_list), len(article_list), len(date_list), len(newspaper_list), '\n')
        page += 1

    print(len(title_list),  len(link_list), len(article_list), len(date_list), len(newspaper_list))
    news_df = pd.DataFrame({'Title': title_list, 'Article': article_list, 'Date': date_list, 'Newspaper': newspaper_list, 'Link': link_list})
    return news_df

In [5]:
def get_article(url):
    driver.get(url)

    html = driver.find_element(By.XPATH, '//*[@id="news_read"]')
    article = cleansing(html.text)
    return article

In [6]:
news_df = news_crawling('035420', 10, False)

page:  1
14 14 14 14 14 

page:  2
26 26 26 26 26 

page:  3
40 40 40 40 40 

page:  4
56 56 56 56 56 

page:  5
66 66 66 66 66 

page:  6
83 83 83 83 83 

page:  7
93 93 93 93 93 

page:  8
107 107 107 107 107 

page:  9
119 119 119 119 119 

page:  10
137 137 137 137 137 

137 137 137 137 137


In [7]:
news_df.iloc[0]['Article']

'美 증시 급락 충격에 2620선 깨졌던 코스피 2620선 회복외국인기관 매도에도 개인 매수하며 지수 추가 하락 방어코스피가 미국 증시 급락의 영향으로 27일 2 가까이 하락하며 출발했다 연합뉴스서울경제국내 증시는 간밤 미국 증시 급락이 부담으로 작용하며 2620선에서 등락을 반복 중이다 외국인과 기관의 매도세에도 개인의 적극적인 순매수세가 지수 추가 하락을 방어하고 있는 것으로 풀이된다27일 오후 1시 26분 기준 코스피는 전날 대비 3398포인트127 내린 263370에 거래 중이다 장 초반 가파른 하락세로 한때 2615선까지 내려왔던 코스피는 다시 2620선을 회복한 후 근처에서 등락을 반복하고 있다투자자별로는 외국인과 기관이 각 3816억 원 2451억 원을 매도하는 동안 개인이 6287억 원을 매수하며 지수를 지탱하고 있다같은 시간 코스피 시총 상위 10개 종목 모두 파란 불이 켜졌다 삼성전자005930는 전날 대비 166 내린 6만 5000원에 거래되며 52주 신저가를 유지 중이며 LG에너지솔루션373220은 전날 대비 248 내린 41만 3000원에 거래 중이다 SK하이닉스000660 NAVER035420 카카오035720가 각 225 279 200 빠지며 2대 넘는 하락세를 보이고 있다장 초반 미국 증시 급락 공포에 지수가 주저앉았지만 개인들의 저가매수세가 확대되며 2620선을 회복하고 있는 것으로 풀이된다 증시 급락의 위기와 공포를 매수 확대의 기회로 삼아야 한다는 의견도 제시된다 이경민 대신증권 연구원은 현재 글로벌 금융시장은 긴축 부담경기 둔화 및 침체 우려 등을 모두 선반영하고 있는 것으로 보인다면서 2600선이 깨진다면 적극적으로 매수해야 할 순간으로 판단된다고 했다 이 연구원은 2600선 아래로 지수가 하락할 가능성도 배제할 수는 없다면서도 코스피 밸류에이션 저점인 12개월 선행 주가수익비율PER 10배를 밑돌기 시작한 이 시점은 공포를 매수로 대응해야 할 때로 보인다고 했다같은 시각 코스닥은 전날 대비 1706포인트187 내린 894

In [8]:
news_df.to_csv('naver.csv', encoding='utf-8', index=False)